# Team 5 - HW4 - Transactions Generator with Stocks #
Data Warehousing - Fall 21

Team 5: Vince Purcell, Brandon Mondile

In [25]:
import csv
from decimal import Decimal
from os import system
import random
import datetime
from datetime import date
import math
import sqlite3 as lite
from sqlite3 import Error

## Global Variables and Functions needed to track SKUs and Inventories ##

In [26]:
def write_record(date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn):
    try:
        
        cur.execute("INSERT INTO Transactions2020 VALUES (?, ?, ?, ?, ?, ?)",
                    (date, customer_number, SKU, price,
                     left_in_stock, cases_bought))
    except lite.OperationalError as err:
        print("insert error: %s", err)


csv.register_dialect('piper', delimiter='|', quoting=csv.QUOTE_NONE)

price_multiplier = 1.2
total_items_bought = 0
customer_count = 0

simulation_length = 365
manufacturer_list = []
product_name_list = []
size_list = []
item_type_list = []
sku_list = []
base_price_list = []
stock_list = []
cases_ordered_list = []
left_in_stock = []
cases_bought = []

milk_price = []
milk_sku = []
milk_stock = []
milk_cases = []
# 858 expected daily sales / 6 milk SKUs * 1.5 days
milk_one_and_one_half = 215

cereal_price = []
cereal_sku = []
cereal_stock = []
cereal_cases = []
# 2250 expected daily sales / 93 cereal SKUs * 3 days
cereal_three_day = 73

baby_food_price = []
baby_food_sku = []
baby_food_stock = []
baby_food_cases = []
# 3457 expected daily sales / 162 baby food SKUs * 3 days
baby_food_three_day = 65

diapers_price = []
diapers_sku = []
diapers_stock = []
diapers_cases = []
# 1911 expected daily sales / 82 diaper SKUs * 3 days
diapers_three_day = 70

bread_price = []
bread_sku = []
bread_stock = []
bread_cases = []
# 1535 expected daily sales / 48 bread SKUs * 3 days
bread_three_day = 96

peanut_butter_price = []
peanut_butter_sku = []
peanut_butter_stock = []
peanut_butter_cases = []
# 534 expected daily sales / 20 pb SKUs * 3 days
peanut_butter_three_day = 81

jelly_jam_price = []
jelly_jam_sku = []
jelly_jam_stock = []
jelly_jam_cases = []
# 234 expected daily sales / 4 jelly_jam SKUs * 3 days
jelly_jam_three_day = 176

random_item_price = []
random_item_sku = []
random_item_stock = []
random_item_cases = []
random_item_three_day = 80

all_items = []
stock_and_cases = []
stock = []

## Read Product list csv and initialize all sku, price, and stock lists ##

Modified leonchuks code to just calculate initial stock based on variables defined above for 3/1.5 day stocks

In [27]:
with open('Products1.txt', 'r') as csvfile:
    for row in csv.DictReader(csvfile, dialect='piper'):
        nosign = row['BasePrice']
        nosign = float(Decimal(nosign.strip('$')))
        nosign = nosign * price_multiplier
        nosign = round(nosign, 2)
        manufacturer_list.append(row.get('Manufacturer'))
        product_name_list.append(row.get('ProductName'))
        size_list.append(row.get('Size'))
        item_type_list.append(row.get('itemType'))
        sku_list.append(row.get('SKU'))
        base_price_list.append(row.get('BasePrice'))
        stock_list.append(row.get('Stock'))
        cases_ordered_list.append(row.get('CasesOrdered'))
        if row['itemType'] == 'Milk':
            milk_price.append(nosign)
            milk_sku.append(row['SKU'])
            cases = math.ceil(milk_one_and_one_half / 12)
            milk_cases.append(cases)
            milk_stock.append(cases * 12)
        elif row['itemType'] == 'Cereal':
            cereal_price.append(nosign)
            cereal_sku.append(row['SKU'])
            cases = math.ceil(cereal_three_day / 12)
            cereal_cases.append(cases)
            cereal_stock.append(cases * 12)
        elif row['itemType'] == 'Baby Food':
            baby_food_price.append(nosign)
            baby_food_sku.append(row['SKU'])
            cases = math.ceil(baby_food_three_day / 12)
            baby_food_cases.append(cases)
            baby_food_stock.append(cases * 12)
        elif row['itemType'] == 'Diapers':
            diapers_price.append(nosign)
            diapers_sku.append(row['SKU'])
            cases = math.ceil(diapers_three_day / 12)
            diapers_cases.append(cases)
            diapers_stock.append(cases * 12)
        elif row['itemType'] == 'Bread':
            bread_price.append(nosign)
            bread_sku.append(row['SKU'])
            cases = math.ceil(bread_three_day / 12)
            bread_cases.append(cases)
            bread_stock.append(cases * 12)
        elif row['itemType'] == 'Peanut Butter':
            peanut_butter_price.append(nosign)
            peanut_butter_sku.append(row['SKU'])
            cases = math.ceil(peanut_butter_three_day / 12)
            peanut_butter_cases.append(cases)
            peanut_butter_stock.append(cases * 12)
        elif row['itemType'] == 'Jelly/Jam':
            jelly_jam_price.append(nosign)
            jelly_jam_sku.append(row['SKU'])
            cases = math.ceil(jelly_jam_three_day / 12)
            jelly_jam_cases.append(cases)
            jelly_jam_stock.append(cases * 12)
        else:
            if (row['itemType'] != 'Milk' and row['itemType'] != 'Cereal' and row['itemType'] != 'Baby Food' and row[
                'itemType'] != 'Diapers' and row['itemType'] != 'Bread' and row['itemType'] != 'Peanut Butter' and row[
                'itemType'] != 'Jelly/Jam'):
                random_item_price.append(nosign)
                random_item_sku.append(row['SKU'])
                cases = math.ceil(random_item_three_day / 12)
                random_item_cases.append(cases)
                random_item_stock.append(cases * 12)

## Functions that return random item by item type for transactions ##

Modified leonchuks functions to do a recursive call when a customer tries to purchase an item out of stock

In [28]:
def get_milk_sku_and_price_stock_cases():
    random_index = random.randrange(len(milk_sku))
    milk_stock[random_index] = milk_stock[random_index] - 1
    if milk_stock[random_index] < 0:
        milk_stock[random_index] = 0
        return get_milk_sku_and_price_stock_cases()
    else:
        return milk_sku[random_index], milk_price[random_index], milk_stock[random_index], milk_cases[random_index]


def get_cereal_sku_and_price_stock_cases():
    random_index = random.randrange(len(cereal_sku))
    cereal_stock[random_index] = cereal_stock[random_index] - 1
    if cereal_stock[random_index] < 0:
        cereal_stock[random_index] = 0
        return get_cereal_sku_and_price_stock_cases()
    else:
        return cereal_sku[random_index], cereal_price[random_index], cereal_stock[random_index], cereal_cases[random_index]


def get_baby_food_sku_and_price_stock_cases():
    random_index = random.randrange(len(baby_food_sku))
    baby_food_stock[random_index] = baby_food_stock[random_index] - 1
    if baby_food_stock[random_index] < 0:
        baby_food_stock[random_index] = 0
        return get_baby_food_sku_and_price_stock_cases()
    else:
        return baby_food_sku[random_index], baby_food_price[random_index], baby_food_stock[random_index], baby_food_cases[random_index]


def get_diapers_sku_and_price_stock_cases():
    random_index = random.randrange(len(diapers_sku))
    diapers_stock[random_index] = diapers_stock[random_index] - 1
    if diapers_stock[random_index] < 0:
        diapers_stock[random_index] = 0
        return get_diapers_sku_and_price_stock_cases()
    else:
        return diapers_sku[random_index], diapers_price[random_index], diapers_stock[random_index], diapers_cases[random_index]


def get_bread_sku_and_price_stock_cases():
    random_index = random.randrange(len(bread_sku))
    bread_stock[random_index] = bread_stock[random_index] - 1
    if bread_stock[random_index] < 0:
        bread_stock[random_index] = 0
        return get_bread_sku_and_price_stock_cases()
    else:
        return bread_sku[random_index], bread_price[random_index], bread_stock[random_index], bread_cases[random_index]


def get_peanut_butter_sku_and_price_stock_cases():
    random_index = random.randrange(len(peanut_butter_sku))
    peanut_butter_stock[random_index] = peanut_butter_stock[random_index] - 1
    if peanut_butter_stock[random_index] < 0:
        peanut_butter_stock[random_index] = 0
        return get_peanut_butter_sku_and_price_stock_cases()
    else:
        return peanut_butter_sku[random_index], peanut_butter_price[random_index], peanut_butter_stock[random_index], peanut_butter_cases[random_index]


def get_jelly_jam_sku_and_price_stock_cases():
    random_index = random.randrange(len(jelly_jam_sku))
    jelly_jam_stock[random_index] = jelly_jam_stock[random_index] - 1
    if jelly_jam_stock[random_index] < 0:
        jelly_jam_stock[random_index] = 0
        return get_jelly_jam_sku_and_price_stock_cases()
    else:
        return jelly_jam_sku[random_index], jelly_jam_price[random_index], jelly_jam_stock[random_index], jelly_jam_cases[random_index]


def get_random_item_sku_and_price_stock_cases():
    random_index = random.randrange(len(random_item_sku))
    random_item_stock[random_index] = random_item_stock[random_index] - 1
    if random_item_stock[random_index] < 0:
        random_item_stock[random_index] = 0
        return get_random_item_sku_and_price_stock_cases()
    else:
        return random_item_sku[random_index], random_item_price[random_index], random_item_stock[random_index], random_item_cases[random_index]

## Stock Order functions ##
from leonchuk code

In [29]:
def order_milk():
    for stock in range(0, len(milk_sku)):
        if milk_stock[stock] < milk_one_and_one_half:
            difference = milk_one_and_one_half - milk_stock[stock]
            cases_needed = math.ceil(difference / 12)
            milk_stock[stock] = milk_stock[stock] + cases_needed * 12
            milk_cases[stock] = milk_cases[stock] + cases_needed


def order_cereal():
    for stock in range(0, len(cereal_sku)):
        if cereal_stock[stock] < cereal_three_day:
            difference = cereal_three_day - cereal_stock[stock]
            cases_needed = math.ceil(difference / 12)
            cereal_stock[stock] = cereal_stock[stock] + cases_needed * 12
            cereal_cases[stock] = cereal_cases[stock] + cases_needed


def order_peanut_butter():
    for stock in range(0, len(peanut_butter_sku)):
        if peanut_butter_stock[stock] < peanut_butter_three_day:
            difference = peanut_butter_three_day - peanut_butter_stock[stock]
            cases_needed = math.ceil(difference / 12)
            peanut_butter_stock[stock] = peanut_butter_stock[stock] + cases_needed * 12
            peanut_butter_cases[stock] = peanut_butter_cases[stock] + cases_needed


def order_jelly():
    for stock in range(0, len(jelly_jam_sku)):
        if jelly_jam_stock[stock] < jelly_jam_three_day:
            difference = jelly_jam_three_day - jelly_jam_stock[stock]
            cases_needed = math.ceil(difference / 12)
            jelly_jam_stock[stock] = jelly_jam_stock[stock] + cases_needed * 12
            jelly_jam_cases[stock] = jelly_jam_cases[stock] + cases_needed


def order_baby_food():
    for stock in range(0, len(baby_food_sku)):
        if baby_food_stock[stock] < baby_food_three_day:
            difference = baby_food_three_day - baby_food_stock[stock]
            cases_needed = math.ceil(difference / 12)
            baby_food_stock[stock] = baby_food_stock[stock] + cases_needed * 12
            baby_food_cases[stock] = baby_food_cases[stock] + cases_needed


def order_diapers():
    for stock in range(0, len(diapers_sku)):
        if diapers_stock[stock] < diapers_three_day:
            difference = diapers_three_day - diapers_stock[stock]
            cases_needed = math.ceil(difference / 12)
            diapers_stock[stock] = diapers_stock[stock] + cases_needed * 12
            diapers_cases[stock] = diapers_cases[stock] + cases_needed


def order_bread():
    for stock in range(0, len(bread_sku)):
        if bread_stock[stock] < bread_three_day:
            difference = bread_three_day - bread_stock[stock]
            cases_needed = math.ceil(difference / 12)
            bread_stock[stock] = bread_stock[stock] + cases_needed * 12
            bread_cases[stock] = bread_cases[stock] + cases_needed


def order_random_items():
    for stock in range(0, len(random_item_sku)):
        if random_item_stock[stock] < random_item_three_day:
            difference = random_item_three_day - random_item_stock[stock]
            cases_needed = math.ceil(difference / 12)
            random_item_stock[stock] = random_item_stock[stock] + cases_needed * 12
            random_item_cases[stock] = random_item_cases[stock] + cases_needed

## Create Grocery DB and Generate Transactions##

I modified leonchuks code to handle a purchase of an out of stock item inside the individual purchase functions above rather than the main script below.

Also fixed bug where the code did not check if max items were hit after each purchase. It originally only checked when buying the random items

In [30]:
try:
    # if grocery.db exists connect to it, otherwise create it
    # one change i want is to have the rocery.db file write into the SQLite folder
    conn = lite.connect('Grocery.db')
except Error as e:
    print("Error %s:" % e.args[0])
    system.exit(1)
finally:
    if conn:
        cur = conn.cursor()
        cur.execute("DROP TABLE IF EXISTS Transactions2020")
        cur.execute("CREATE TABLE Transactions2020 (date_text TEXT, customer_number INT, SKU TEXT, price FLOAT, "
                    "left_in_stock INT, cases_bought INT)")

customers_low = 1020
customers_high = 1060
weekend_increase = 50
maximum_items = 80
simulation_start_date = date(2020, 1, 1)
simulation_end_date = date(2020, 12, 31)
current_date = simulation_start_date
daily_customers = 0
my_items = 0
customer_number = 1

for iday in range(0, simulation_length):
    if (( (iday+1) % 20)==0):
        print("Working on Day ", iday + 1)
    increase = 0
    current_date += datetime.timedelta(1)
    if current_date.weekday() >= 5:
        increase = weekend_increase

    order_milk()

    if current_date.weekday() == 1 or current_date.weekday() == 3 or current_date.weekday() == 5:
        order_cereal()
        order_baby_food()
        order_bread()
        order_diapers()
        order_jelly()
        order_peanut_butter()
        order_random_items()

    daily_customers = random.randint(customers_low + increase, customers_high + increase)
    customer_number = 1

    while customer_number <= daily_customers:
        customer_count = customer_count + 1
        my_items = random.randint(1, maximum_items)
        k = 0
        if random.randint(1, 100) <= 70:
            SKU, price, left_in_stock, cases_bought = get_milk_sku_and_price_stock_cases()
            write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
            k += 1

            if random.randint(1, 100) <= 50 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_cereal_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        else:
            if random.randint(1, 100) <= 5 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_cereal_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        if random.randint(1, 100) <= 20 and k < my_items:
            SKU, price, left_in_stock, cases_bought = get_baby_food_sku_and_price_stock_cases()
            write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
            k += 1

            if random.randint(1, 100) <= 80 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_diapers_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        else:
            if random.randint(1, 100) <= 1 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_diapers_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        if random.randint(1, 100) <= 50 and k < my_items:
            SKU, price, stock, cases = get_bread_sku_and_price_stock_cases()
            write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
            k += 1

        if random.randint(1, 100) <= 10 and k < my_items:
            SKU, price, left_in_stock, cases_bought = get_peanut_butter_sku_and_price_stock_cases()
            write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
            k += 1

            if random.randint(1, 100) <= 90 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_jelly_jam_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        else:
            if random.randint(1, 100) <= 5 and k < my_items:
                SKU, price, left_in_stock, cases_bought = get_jelly_jam_sku_and_price_stock_cases()
                write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
                k += 1

        while k < my_items:
            SKU, price, left_in_stock, cases_bought = get_random_item_sku_and_price_stock_cases()
            write_record(current_date, customer_number, SKU, price, left_in_stock, cases_bought, cur, conn)
            k += 1

        customer_number = customer_number + 1
    conn.commit()
print("Working on Day ", iday + 1)
conn.close()

Working on Day  20
Working on Day  40
Working on Day  60
Working on Day  80
Working on Day  100
Working on Day  120
Working on Day  140
Working on Day  160
Working on Day  180
Working on Day  200
Working on Day  220
Working on Day  240
Working on Day  260
Working on Day  280
Working on Day  300
Working on Day  320
Working on Day  340
Working on Day  360
Working on Day  365


## Create Table with SKUs and Item names ##
This table will be used for item name lookup by SKU

In [31]:
csv.register_dialect('piper', delimiter= '|', quoting = csv.QUOTE_NONE)

con = lite.connect(r'Grocery.db')

cur = con.cursor()

cur.execute('drop table if exists itemLookup')
cur.execute('create Table itemLookup(itemstr TEXT, sku INT)')

with open('Products1.txt', 'r') as csvfile:
    for row in csv.DictReader(csvfile, dialect= 'piper'):
        sku = int(row['SKU'])
        itemstr = str(row['Manufacturer']) + " " + str(row['Product Name']) + " (" + str(row['itemType']) + ")"
        try:
            cur.execute("INSERT INTO itemLookup VALUES (?,?)",
                        (itemstr, sku))
        except lite.OperationalError as err:
            print("insert error: %s", err)
            break

con.commit()
con.close()

## Count Total Customers and Sales ##

In [32]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()
s = 'select count(customer_number) from Transactions2020'
cur.execute(s)
query = cur.fetchall()
total_items = query[0][0]
con.close()

## Get count of most common SKUs and print summary ##

In [33]:
print("Summary\n----------------------------------------------------")
print("Customer Count and Total Sales: " + str(customer_count) + "\n")
print("Total items bought: " + str(total_items) + "\n")

con = lite.connect(r'Grocery.db')
cur = con.cursor()
s = 'select count(SKU) as cnt1, SKU from Transactions2020 group by SKU order by cnt1 DESC limit 10'
cur.execute(s)
rows = cur.fetchall()
print("Most Popular Items by SKU")
print('Count - SKU      - Item Name')
for row in rows:
    s = 'select itemstr from itemLookup where sku like ' + str(row[1])
    cur.execute(s)
    item = cur.fetchall()
    print(str(row[0]) + " - " + str(row[1]) + " - " + item[0][0])
con.close()

Summary
----------------------------------------------------
Customer Count and Total Sales: 384772

Total items bought: 15586657

Most Popular Items by SKU
Count - SKU      - Item Name
44900 - 42355001 - Rowan Dairy 1.00% Milk (Milk)
44891 - 42360001 - Rowan Dairy Whole Milk Milk (Milk)
44855 - 42359001 - Rowan Dairy Whole Milk Milk (Milk)
44807 - 42358001 - Rowan Dairy 2.00% Milk (Milk)
44697 - 42357001 - Rowan Dairy 2.00% Milk (Milk)
44692 - 42356001 - Rowan Dairy 1.00% Milk (Milk)
12657 - 42311001 - Smuckers Jam Grape (Jelly/Jam)
12651 - 42312001 - Smuckers Jam Strawberry (Jelly/Jam)
12548 - 42314001 - Smuckers Squeeze Jelly Grape (Jelly/Jam)
12505 - 42313001 - Smuckers Jelly Grape (Jelly/Jam)


# Code Below was taken from the code provided by Professor Brietzman #
Code runs analysis on generated transactions to assure generation script has no bugs

In [34]:
#a couple of handy functions taken from
#https://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html#printing-a-database-summary
def total_rows(cursor, table_name, print_out=False):
    """ Returns the total number of rows in the database """
    cursor.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cursor.fetchall()
    if print_out:
        print('\nTotal rows: {}'.format(count[0][0]))
    return count[0][0]


def table_col_info(cursor, table_name, print_out=False):
    """ Returns a list of tuples with column informations:
    (id, name, type, notnull, default_value, primary_key)
    """
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()

    if print_out:
        print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
        for col in info:
            print(col)
    return info

In [35]:
con = lite.connect('Grocery.db')
cur = con.cursor()

In [36]:
def strip_uni(s):
      return s.replace(u'\xa0', u' ')

cur.execute("DROP TABLE IF EXISTS products")
s = 'CREATE TABLE products(Manufacturer varchar(255), Product Name varchar(255), Size varchar(255),'
s = s+' itemType varchar(255),SKU varchar(8), BasePrice float)'
cur.execute(s)

i = 0
with open('Products1.txt', 'r') as csvfile:
    for row in csv.DictReader(csvfile, dialect= 'piper'):
        i = i + 1
        try:
              cur.execute("INSERT INTO products VALUES (?,?,?,?,?,?)",
                          (row['Manufacturer'],row['Product Name'],strip_uni(row['Size']),
                           row['itemType'],row['SKU'],float(row['BasePrice'][1:])))
        except lite.OperationalError as err:
                print("insert error: %s", err)
        if ((i % 1000)==0):
            print("inserted row ",str(i))
            con.commit()
            
csvfile.close()
con.commit()

inserted row  1000
inserted row  2000


In [37]:
#how many rows?
total_rows(cur,'Transactions2020',False)

15586657

In [38]:
#looks like they passed test 1.  Now let's count the customers per day
s = "CREATE TEMP VIEW Vcust as "
s = s + "SELECT date_text, max(customer_number) as maxTnum from Transactions2020 group by date_text;"
cur.execute(s)

In [39]:
#look at the first 2 weeks
cur.execute("SELECT * FROM Vcust Limit 14" )
rows = cur.fetchall()
for row in rows:
    print(row)

('2020-01-02', 1033)
('2020-01-03', 1055)
('2020-01-04', 1075)
('2020-01-05', 1072)
('2020-01-06', 1049)
('2020-01-07', 1052)
('2020-01-08', 1048)
('2020-01-09', 1037)
('2020-01-10', 1048)
('2020-01-11', 1095)
('2020-01-12', 1088)
('2020-01-13', 1032)
('2020-01-14', 1050)
('2020-01-15', 1056)


In [40]:
#Looks like 1020 to 1060 with +50 on weekends.  So far so good.
#just to be sure we'll look at the average over 365 days
cur.execute("SELECT avg(maxTnum) as avgCust from Vcust" )
rows = cur.fetchall()
for row in rows:
    print(row)

(1054.1698630136987,)


In [41]:
def print_select(cur, select_statement, max_rows):
    select_statement = select_statement + " limit "+str(max_rows)
    cur.execute(select_statement)
    rows = cur.fetchall()
    for row in rows:
      print(row)

### Check the number of items per customer ###
### Each customer bought between 1 and 80 and it should be equally likely ###

In [42]:
s = "CREATE TEMP VIEW Vitems as "
s = s + "SELECT date_text, customer_number,count(SKU) as count1 from Transactions2020 group by date_text,customer_number;"
cur.execute(s)
print_select(cur, "SELECT * FROM Vitems",10)

('2020-01-02', 1, 63)
('2020-01-02', 2, 58)
('2020-01-02', 3, 3)
('2020-01-02', 4, 58)
('2020-01-02', 5, 68)
('2020-01-02', 6, 48)
('2020-01-02', 7, 58)
('2020-01-02', 8, 33)
('2020-01-02', 9, 34)
('2020-01-02', 10, 44)


### Each value of 1 to 80 should be equally likely ###

In [43]:
s = "Select count1 as numItemsPerCust, count(customer_number) as total from Vitems group by count1"
print_select(cur,s,500)

(1, 4809)
(2, 4733)
(3, 4779)
(4, 4768)
(5, 4923)
(6, 4848)
(7, 4673)
(8, 4795)
(9, 4828)
(10, 4726)
(11, 4947)
(12, 4755)
(13, 4769)
(14, 4860)
(15, 4776)
(16, 4681)
(17, 4883)
(18, 4832)
(19, 4732)
(20, 4762)
(21, 4902)
(22, 4834)
(23, 4785)
(24, 4836)
(25, 4841)
(26, 4775)
(27, 4865)
(28, 4795)
(29, 4803)
(30, 4917)
(31, 4783)
(32, 4812)
(33, 4908)
(34, 4772)
(35, 4884)
(36, 4888)
(37, 4775)
(38, 4700)
(39, 4907)
(40, 4717)
(41, 4933)
(42, 4832)
(43, 4795)
(44, 4775)
(45, 4774)
(46, 4876)
(47, 4904)
(48, 4801)
(49, 4678)
(50, 4732)
(51, 4840)
(52, 4788)
(53, 4736)
(54, 4765)
(55, 4801)
(56, 4701)
(57, 4814)
(58, 4929)
(59, 4735)
(60, 4792)
(61, 4853)
(62, 4767)
(63, 4854)
(64, 4810)
(65, 4939)
(66, 4917)
(67, 4782)
(68, 4934)
(69, 4774)
(70, 4763)
(71, 4863)
(72, 4762)
(73, 4874)
(74, 4839)
(75, 4882)
(76, 4747)
(77, 4767)
(78, 4722)
(79, 4725)
(80, 4819)


In [44]:
# Create products table
cur.execute("DROP TABLE IF EXISTS products")
s = 'CREATE TABLE products(Manufacturer varchar(255), Product Name varchar(255), Size varchar(255),'
s = s+' itemType varchar(255),SKU varchar(8), BasePrice float)'
cur.execute(s)

## Create Product Table ##

In [45]:
def strip_uni(s):
      return s.replace(u'\xa0', u' ')

cur.execute("DROP TABLE IF EXISTS products")
s = 'CREATE TABLE products(Manufacturer varchar(255), Product Name varchar(255), Size varchar(255),'
s = s+' itemType varchar(255),SKU varchar(8), BasePrice float)'
cur.execute(s)

i = 0
with open('Products1.txt', 'r') as csvfile:
    for row in csv.DictReader(csvfile, dialect= 'piper'):
        i = i + 1
        try:
              cur.execute("INSERT INTO products VALUES (?,?,?,?,?,?)",
                          (row['Manufacturer'],row['Product Name'],strip_uni(row['Size']),
                           row['itemType'],row['SKU'],float(row['BasePrice'][1:])))
        except sqlite3.OperationalError as err:
                print("insert error: %s", err)
        if ((i % 1000)==0):
            print("inserted row ",str(i))
            con.commit()
            
csvfile.close()
con.commit()

inserted row  1000
inserted row  2000


## Top selling products ##

In [46]:
s='Create TEMP View vTopSKUs as Select count(customer_number) as TotalSold, SKU from Transactions2020 group by SKU Order by TotalSold DESC;'
cur.execute(s)
print_select(cur,'Select * from vTopSKUs',20)

(44900, '42355001')
(44891, '42360001')
(44855, '42359001')
(44807, '42358001')
(44697, '42357001')
(44692, '42356001')
(12657, '42311001')
(12651, '42312001')
(12548, '42314001')
(12505, '42313001')
(9228, '42847001')
(9181, '42378001')
(9180, '42122001')
(9177, '43782001')
(9163, '43506001')
(9150, '43624001')
(9145, '42375001')
(9139, '42452001')
(9137, '42765001')
(9135, '42337001')


In [47]:
s = 'Select sum(TotalSold) as Total, Products.itemType as ItemType from vTopSKUs inner join Products on '
s = s+'vTopSKUs.SKU=Products.SKU group by ItemType Order by Total DESC'
print_select(cur,s,50)

(2614292, '')
(1477606, 'Snacks')
(1183186, 'Coffee/Creamer')
(1066934, 'Fresh Vegetables')
(676126, 'Baked Goods Other than Bread')
(623034, 'Fresh Fruit')
(524373, 'Pasta/Noodles')
(401041, 'Soup')
(400091, 'Cheese')
(355759, 'Cookies')
(302393, 'Frozen Food')
(268842, 'Milk')
(267341, 'Canned Vegetables')
(223229, 'Frozen Vegetables')
(222917, 'Chocolate Candy')
(221729, 'Nuts')
(188769, 'Bread')
(186248, 'Canned Goods')
(178254, 'Dips')
(177742, 'Cooking Oil')
(177601, 'Popsicles')
(177583, 'Deli Meats')
(177362, 'Pizza')
(138361, 'Cereal')
(134117, 'Spices')
(133760, 'Bologna')
(133512, 'Deli Salads')
(133424, 'Popcorn')
(133337, 'French Fries')
(133210, 'TV Dinner')
(132969, 'Cleaners')
(132896, 'Frozen Chicken')
(124047, 'Hot Dogs')
(106718, 'Soda/Juice/Drinks')
(106577, 'Rice')
(97961, 'Yogurt')
(97703, 'Gravy/Sauce')
(89180, 'Sugar')
(89094, 'Ice Cream')
(89055, 'Aspirin')
(88970, 'Hard Candy')
(88947, 'Hamburger')
(88659, 'Canned Fruit')
(88617, 'Cottage Cheese')
(88520, 'Tun

In [48]:
con.close()